In [1]:
#load ehr_stream and target_values from pickle files
import pickle
from configuration import FilePaths
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
FILE_PATHS = FilePaths()

ehr_stream = pickle.load(open(FILE_PATHS.EHR_STREAM, "rb"))
target_values = pickle.load(open(FILE_PATHS.TARGET_VALUES, "rb"))

In [84]:
df = pd.DataFrame(ehr_stream)

# Sample 

import pprint
pprint.pprint(f'Input timestream {df.loc[1290]}')
pprint.pprint(f'Will they revisit: {target_values[1290]}')

('Input timestream 0     Recicort Ear Drops ; Consultation - Repeat ; D...\n'
 '1                                                     1\n'
 '2                                                     2\n'
 '3                                                     3\n'
 '4                                                     4\n'
 '5                                                     5\n'
 '6                                                     6\n'
 '7                                                     7\n'
 '8                                                     8\n'
 '9                                                     9\n'
 '10                                                   10\n'
 '11                                                   11\n'
 '12                                                   12\n'
 '13                                                   13\n'
 '14                                                   14\n'
 '15                                                   15\n'
 '16   

In [3]:
def tokenize(column):
    """Tokenizes a Pandas dataframe column and returns a list of tokens.

    Args:
        column: Pandas dataframe column (i.e. df['text']).

    Returns:
        tokens (list): Tokenized list, i.e. [Donald, Trump, tweets]

    """

    tokens = nltk.word_tokenize(column)
    return [w for w in tokens]   

In [4]:
def remove_stopwords(tokenized_column):
    """Return a list of tokens with English stopwords removed. 

    Args:
        column: Pandas dataframe column of tokenized data from tokenize()

    Returns:
        tokens (list): Tokenized list with stopwords removed.

    """
    stops = set(stopwords.words("english"))
    return [word for word in tokenized_column if not word in stops]

In [5]:
def rejoin_words(tokenized_column):
    """Rejoins a tokenized word list into a single string. 
    
    Args:
        tokenized_column (list): Tokenized column of words. 
        
    Returns:
        string: Single string of untokenized words. 
    """
    return ( " ".join(tokenized_column))

In [6]:
# loop through dataframe columns and tokenize
for column in df.columns:
    df[column] = df[column].apply(tokenize)
    df[column] = df[column].apply(remove_stopwords)
    df[column] = df[column].apply(rejoin_words)


In [64]:
df['targets'] = target_values

#issue with pads returning true for all targets
df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,targets
0,DHPPi vaccination ; Leptospira vaccination His...,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,True
1,Reason : Sore Ear Appointment Notes : Overdue ...,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,True
2,HPC 6 Month Check In 6 month HPC check BAR EDU...,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,True
3,Nobivac Lepto 4 50D ; Endectrid Dog Large 10 k...,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,True
4,Leptospira vaccination Reason : Annual Health ...,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3756,sonotix ear cleaner sol 120ml Use clean ears <...,1,2,3,4,postop check up- concerns vomitted night surge...,1,2,3,4,...,16,17,18,19,20,21,22,23,24,True
3757,postop check up- concerns vomitted night surge...,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,True
3758,Consultation - Initial season usual timings st...,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,True
3759,Cerenia Inj 10mg/ml ( dog ) ; injection fee ; ...,1,2,3,4,5,6,Rcw Gastro Intest Canine Wet 12x400gm O report...,1,2,...,14,15,16,17,18,19,20,21,22,True


In [85]:
from sklearn.model_selection import train_test_split



X_train,X_test,y_train,y_test = train_test_split(df.index,target_values,test_size=0.2)
df.iloc[X_train]


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [87]:
df.iloc[X_train]

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
2171,"BAR, DUDE, O. No concern\r\nHeart no murmur an...",1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
470,Quantex Dog & Cat Tablet ; Vaccinate Dog - Sec...,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
548,rycarfa tabs 20mg (pk100) per tab Dose HALF a ...,1,2,3,4,5,6,consultation post op check Wound fully healed ...,1,2,...,13,14,15,16,17,18,19,20,21,22
1325,Kennel Cough vaccination Examination: BAR and ...,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
655,Re-check / Repeat / Recall / Medical Progress ...,1,2,3,4,5,Reason: Recheck\r\nAppointment Notes: Overdue ...,1,2,3,...,Re-check / Repeat / Recall / Medical Progress ...,1,2,3,4,5,6,7,8,Reason: Poc & Stitches Out - Joe\r\nAppointmen...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,Sick/Trauma Consult Reason: Vomiting After Dri...,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
2416,Reason: Booster Vaccination and Health Exam/ve...,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
1213,Dispensing Fee Tablets ; Dispensing Fee Tablet...,1,Post Operative Check - No Problem Flush wound ...,1,2,3,4,5,6,7,...,18,19,20,21,22,23,24,25,26,27
1772,consultation post op check In for 1st post op ...,1,2,3,4,5,6,dispensing fee ; prednicare tabs 5mg (500pk)...,1,2,...,13,14,15,16,17,18,19,20,21,22
